<a href="https://colab.research.google.com/github/Mamtajangra/pytorch_ANN_model/blob/main/ANN_fashion_mnist_optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt

In [31]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [32]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [2]:
df = pd.read_csv("/content/fashion-mnist_train.csv")
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0,0,0,0,0,0,0,5,0,...,0.0,0.0,0.0,30.0,43.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,2,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1882,8,0,0,0,0,0,0,0,0,0,...,217.0,216.0,215.0,218.0,203.0,5.0,0.0,2.0,0.0,0.0
1883,6,0,0,0,0,0,0,0,0,0,...,71.0,62.0,64.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1884,4,0,0,0,0,0,0,0,0,2,...,6.0,0.0,55.0,210.0,210.0,133.0,0.0,0.0,0.0,0.0
1885,5,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df.head(4)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0,0,0,0,0,0,0,5,0,...,0.0,0.0,0.0,30.0,43.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,2,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [4]:
df.tail(3)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
1884,4,0,0,0,0,0,0,0,0,2,...,6.0,0.0,55.0,210.0,210.0,133.0,0.0,0.0,0.0,0.0
1885,5,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1886,6,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Columns: 785 entries, label to pixel784
dtypes: float64(470), int64(315)
memory usage: 11.3 MB


In [6]:
df.shape

(1887, 785)

In [7]:
df.describe()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
count,1887.000000,1887.000000,1887.000000,1887.000000,1887.000000,1887.000000,1887.000000,1887.000000,1887.000000,1887.000000,...,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000,1886.000000
mean,4.366190,0.007419,0.011659,0.035506,0.129306,0.240064,0.357711,0.558029,2.001060,5.387917,...,35.554613,23.498409,16.691941,18.425769,23.951750,19.159597,9.252386,2.999470,0.750265,0.036055
std,2.845432,0.322286,0.279887,0.355700,2.356873,3.387508,4.030737,5.435703,12.491779,22.989394,...,58.441891,48.860531,42.639915,44.505495,53.052682,46.660436,30.482102,17.545893,7.882942,0.779563
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,60.000000,12.000000,0.000000,0.750000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.000000,14.000000,10.000000,10.000000,87.000000,88.000000,114.000000,121.000000,185.000000,216.000000,...,248.000000,231.000000,251.000000,243.000000,245.000000,239.000000,234.000000,217.000000,192.000000,30.000000


In [13]:
df.isnull().sum()

,0
label,0
pixel1,0
pixel2,0
pixel3,0
pixel4,0
...,...
pixel780,1
pixel781,1
pixel782,1
pixel783,1


In [16]:
df = df.fillna(df.mean(numeric_only = True))

In [17]:
df.isnull().sum()

,0
label,0
pixel1,0
pixel2,0
pixel3,0
pixel4,0
...,...
pixel780,0
pixel781,0
pixel782,0
pixel783,0


In [18]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [20]:
X_train = X_train/255.0
X_test = X_test/255.0

In [23]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    # Convert to PyTorch tensors
    self.features = torch.tensor(features.values, dtype=torch.float32)
    self.labels = torch.tensor(labels.values, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [24]:
train_dataset = CustomDataset(X_train, y_train)

In [25]:
test_dataset = CustomDataset(X_test, y_test)

In [26]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [27]:
len(train_loader)

48

In [28]:
class MyNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.BatchNorm1d(128),  ## remove overfitting via batchnormalization means normalize the output comes from activation function
            nn.ReLU(),
            nn.Dropout(p=0.3),      ## it is another method to overcome overfitting such that it blocks some neuron and get result easily

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.model(x)

In [29]:
learning_rate = 0.01
epochs = 100

In [33]:
model = MyNN(X_train.shape[1])

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)  ## l2 regularization it is another method to overcome overfitting with the addition of some penality
                                  #  this decay is the value of lambda like loss = loss(original) + lambda(w**2)

In [34]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


Epoch: 1 , Loss: 1.2927059506376584
Epoch: 2 , Loss: 0.8489935745795568
Epoch: 3 , Loss: 0.7590589467436075
Epoch: 4 , Loss: 0.6919467244297266
Epoch: 5 , Loss: 0.6224376745522022
Epoch: 6 , Loss: 0.5650100819766521
Epoch: 7 , Loss: 0.5481360405683517
Epoch: 8 , Loss: 0.5453747982780138
Epoch: 9 , Loss: 0.48225164289275807
Epoch: 10 , Loss: 0.48217856387297314
Epoch: 11 , Loss: 0.4654444806898634
Epoch: 12 , Loss: 0.46869179389129084
Epoch: 13 , Loss: 0.4397988210742672
Epoch: 14 , Loss: 0.4049964065973957
Epoch: 15 , Loss: 0.44775151709715527
Epoch: 16 , Loss: 0.4307700792948405
Epoch: 17 , Loss: 0.4073227693637212
Epoch: 18 , Loss: 0.4146229913458228
Epoch: 19 , Loss: 0.36711144105841714
Epoch: 20 , Loss: 0.36650137106577557
Epoch: 21 , Loss: 0.4065231929222743
Epoch: 22 , Loss: 0.3912150909503301
Epoch: 23 , Loss: 0.38291447547574836
Epoch: 24 , Loss: 0.35469220563148457
Epoch: 25 , Loss: 0.3921636464074254
Epoch: 26 , Loss: 0.3599105275546511
Epoch: 27 , Loss: 0.3231872290683289
Ep

In [35]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [38]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.671957671957672


In [39]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8508946322067594
